In [139]:
import pandas as pd 
import numpy as np 
import os 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import zscore
from sklearn.decomposition import FastICA, PCA
os.makedirs(os.path.join(os.getcwd(),'images'),exist_ok=True)
os.makedirs(os.path.join(os.getcwd(),'datasets'),exist_ok=True)
IMAGE_DIR = os.path.join(os.getcwd(),'images')
DATASETS = os.path.join(os.getcwd(),'datasets')
import matlab.engine


In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()

# Add path to where your script is
eng.addpath(r'/home/volpym/Dropbox/Notes/MSC/Biosignal Processing - Neuroinformatics/projects')

print("jadeR exists:", eng.exist('jadeR'))

jadeR exists: 2.0


In [3]:
data = np.loadtxt(os.path.join(os.getcwd(),'datasets','FOETAL_ECG.dat')) 
signal_name = 'foetal'
df = pd.DataFrame(data)
time = df[0].values
df = df.drop(0,axis=1)
ecg_df = pd.DataFrame(zscore(df, axis=0))

channel_names = [f"channel_{i+1}" for i in range(ecg_df.shape[1])]
ecg_df.columns = channel_names

fig = make_subplots(rows=len(channel_names), cols=2, subplot_titles=["Original Traces", 'estimated source-signals'])

offset = 2500 # vertical spacing between channels
scale = 100

for i, channel in enumerate(channel_names):
    fig.add_trace(
        go.Scatter(
            x=ecg_df.index,
            y=ecg_df[channel].values,
            mode='lines',
            line=dict(width=1),
            name=f'Ch{i+1}',
            showlegend=False
        ),
        row=i+1, col=1
    )

eng.workspace['ECGdata'] = matlab.double(df.values.T.tolist())


# Get variable back from MATLAB workspace
b = eng.jadeR(eng.workspace['ECGdata'])
b = np.array(b)
#ecg_df = pd.DataFrame(zscore(pd.DataFrame(b), axis=0))

sources =np.dot(b, df.values.T)
source_df = pd.DataFrame(sources)
sources_df = pd.DataFrame(zscore(source_df.T, axis=0))
sources_df.columns = channel_names
for i, channel in enumerate(channel_names):
    fig.add_trace(
        go.Scatter(
            x=sources_df.index,
            y=sources_df[channel].values, 
            mode='lines',
            line=dict(width=1),
            name=f'Ch{i+1}',
            showlegend=False
        ),
        row=i+1, col=2
    )

fig.update_layout(
    height=1000,
    width=1600,
    title="Original ECG Traces",
    xaxis_title='Time (s)',
    template='plotly_white'
    
)
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True,)
#fig.write_image(os.path.join(IMAGE_DIR,f"{signal_name}_original_trace.png"), width=800, height=500, scale=2)
fig.show()

jade -> Looking for 8 sources
jade -> Removing the mean value
jade -> Whitening the data
jade -> Estimating cumulant matrices
jade -> Initialization of the diagonalization
jade -> Contrast optimization by joint diagonalization
jade -> Sweep #0
jade -> Sweep #1
jade -> Sweep #2
jade -> Sweep #3
jade -> Sweep #4
jade -> Sweep #5
jade -> Sweep #6
jade -> Total of 114 Givens rotations
jade -> Sorting the components
jade -> Fixing the signs


In [4]:
sources_df

,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8
0,0.207022,0.290749,0.913854,1.054256,0.337468,-0.946184,-1.283422,-0.650484
1,0.779578,0.839899,0.910032,0.113527,0.401112,-1.629348,-1.051936,-0.416093
2,0.238150,0.266266,0.655722,0.838140,0.925293,-1.016066,-0.615827,-1.097861
3,0.080851,-0.188013,-0.133338,0.085846,1.736539,-1.869641,-0.146328,-0.052251
4,0.368872,0.334724,0.369059,0.007746,1.148550,-1.336864,-0.009798,-0.472949
...,...,...,...,...,...,...,...,...
2495,-0.009965,0.294535,0.127638,1.260835,-0.425546,0.883180,-0.027374,-1.546392
2496,0.051337,0.438966,0.309366,0.543946,-1.062341,0.906935,0.072114,-1.239855
2497,0.365334,0.995143,0.770996,0.306666,-1.704323,0.773642,-0.058106,-1.334431
2498,0.229923,0.922671,0.897831,0.523494,-1.821185,0.763789,0.095071,-1.363295


display('selecting the 1st and 7th ICs from the JADE-ICA method')
figure,plot(time,zscore(Sources(1,:)),time,zscore(Sources(7,:))+6),grid,xlabel('time'),legend('1st','7th')
title('selected ICs from JADE-ICA method')


In [15]:
fig = make_subplots(rows=len(sources_df[['channel_1','channel_7']].columns.tolist()), cols=1, subplot_titles=["Original Traces", 'estimated source-signals'])

for i, channel in enumerate(sources_df[['channel_1','channel_7']].columns.tolist()):
    print(channel)
    fig.add_trace(
        go.Scatter(
            x=time,
            y=sources_df[channel].values, 
            mode='lines',
            line=dict(width=1),
            name=f'Ch{i+1}',
            showlegend=False
        ),
        row=i+1, col=1
    )

fig.update_layout(
    height=1000,
    width=1600,
    title="Selected ICs from JADE-ICA method",
    xaxis_title='Time (s)',
    template='plotly_white'
    
)
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True,)
#fig.write_image(os.path.join(IMAGE_DIR,f"{signal_name}_original_trace.png"), width=800, height=500, scale=2)
fig.show()

channel_1
channel_7


In [133]:
fig = make_subplots(rows=len(ecg_df.columns.tolist()), cols=2, subplot_titles=["Original Traces", 'FASTICA based estimated source-signals'])

offset = 2500 # vertical spacing between channels
scale = 100

for i, channel in enumerate(channel_names):
    
    fig.add_trace(
        go.Scatter(
            x=ecg_df.index,
            y=ecg_df[channel].values,
            mode='lines',
            line=dict(width=1),
            name=f'Ch{i+1}',
            showlegend=False
        ),
        row=i+1, col=1
    )
ica = FastICA(n_components=3)
S = ica.fit_transform(ecg_df.values)   # sources (icasig)
A = ica.mixing_            # mixing matrix (A)
W = ica.components_ 

fica_df = pd.DataFrame(S)
fica_df = pd.DataFrame(zscore(fica_df, axis=0))
for i, channel in enumerate(fica_df.columns.tolist()):
    fig.add_trace(
        go.Scatter(
            x=time,
            y=fica_df[channel].values, 
            mode='lines',
            line=dict(width=1),
            name=f'Ch{i+1}',
            showlegend=False
        ),
        row=i+1, col=2
    )

fig.update_layout(
    height=1000,
    width=1600,
    title="Original ECG Traces",
    xaxis_title='Time (s)',
    template='plotly_white'
    
)
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True,)
fig.write_image(os.path.join(IMAGE_DIR,f"fastica_foetal.png"), width=800, height=500, scale=2)
fig.show()